# MNIST Digit Recognition Exercise

This notebook covers:
1. Reproducing digit recognition for two digits.
2. Extending the network to all 10 digits.
3. Improving performance using deeper networks and Adam optimizer.

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Part A: Two-digit classification

In [ ]:
# Load MNIST dataset, filter only digits 0 and 1
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST('.', train=True, download=True, transform=transform)
test_data = datasets.MNIST('.', train=False, download=True, transform=transform)

train_idx = [i for i, t in enumerate(train_data.targets) if t in [0,1]]
test_idx = [i for i, t in enumerate(test_data.targets) if t in [0,1]]

train_subset = Subset(train_data, train_idx)
test_subset = Subset(test_data, test_idx)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=1000, shuffle=False)

# Define simple neural network
class TwoDigitNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 1)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = TwoDigitNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

## Part B: Extend to 10 digits

In [ ]:
# Full dataset loader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

# Neural network for 10-class classification
class TenDigitNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # logits
        return x

model = TenDigitNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

## Part C: Improve performance with more layers and Adam

In [ ]:
class DeepNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DeepNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop template
def train(model, device, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Example: train(model, device, train_loader, optimizer, criterion)